In [ ]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 48.9 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import requests
from github import Github
import re
from datasets import Dataset
import os


In [ ]:
# initialize PyGithub with the GitHub token
g = Github("######")

# specify the repository
repo = g.get_repo("openai/gym")

In [ ]:
# function to extract Python functions from a script
def extract_functions_from_code(code):
    pattern = re.compile(r"def\s+(\w+)\s*\(.*\):")
    functions = pattern.findall(code)
    return functions

# fetch Python files from the repository
python_files = []
contents = repo.get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    elif file_content.path.endswith(".py"):
        python_files.append(file_content)

# extract functions and create dataset
data = {"code": [], "function_name": []}
for file in python_files:
    code = file.decoded_content.decode("utf-8")
    functions = extract_functions_from_code(code)
    for function in functions:
        data["code"].append(code)
        data["function_name"].append(function)



In [ ]:
# create a Hugging Face dataset
dataset = Dataset.from_dict(data)

# save the dataset to disk
dataset.save_to_disk("/content/drive/MyDrive/code_generation_dataset")

print("Dataset created and saved to disk.")

Saving the dataset (0/1 shards):   0%|          | 0/974 [00:00<?, ? examples/s]

Dataset created and saved to disk.


Training Model

In [40]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [41]:
# set the pad_token to eos_token or add a new pad token
tokenizer.pad_token = tokenizer.eos_token

# load the dataset
dataset = load_from_disk("/content/drive/MyDrive/code_generation_dataset")

# split the dataset into training and test sets
dataset = dataset.train_test_split(test_size=0.1)

In [42]:
dataset

DatasetDict({
    train: Dataset({
        features: ['code', 'function_name'],
        num_rows: 876
    })
    test: Dataset({
        features: ['code', 'function_name'],
        num_rows: 98
    })
})

In [57]:
# preprocess the dataset
def preprocess_function(examples):
    # Tokenize the code
    tokenized_code = tokenizer(examples['code'], truncation=True, padding='max_length', max_length=512)

    # Create labels by shifting the input_ids
    tokenized_code['labels'] = tokenized_code['input_ids'].copy()  # Labels are the same as input_ids for Causal LM

    return tokenized_code

In [58]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/876 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [59]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['code', 'function_name', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 876
    })
    test: Dataset({
        features: ['code', 'function_name', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 98
    })
})

In [60]:
# fine-tune the model
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

In [61]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=219, training_loss=0.32252519860115225, metrics={'train_runtime': 221.8455, 'train_samples_per_second': 3.949, 'train_steps_per_second': 0.987, 'total_flos': 818849205190656.0, 'train_loss': 0.32252519860115225, 'epoch': 1.0})

In [62]:
# prompt: save the trained model

trainer.save_model("/content/drive/MyDrive/Results/code_generation_trained_model")

In [66]:
prompt = "Write a Python program for add two numbers."
inputs = tokenizer(prompt, return_tensors="pt")
inputs=inputs.to(model.device)
outputs = model.generate(inputs["input_ids"], max_length=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Write a Python program for add two numbers.

This program is based on the "Exercise 7" assignment given by
Andrew Klimov

### Description
The program is provided with two integers, and the program will add the two integers
to the current integer.

### Example
The first integer is 3, the second is 4, and the sum is 6.

### Output
3 + 4 = 6

### Hint
This problem is very similar to the exercise 7, but now the number system is
assumed to be the same as in the previous exercise.

### Notes
This problem has a few important properties:

* The number system is not the same as the default


In [69]:
prompt = "write a python program for checking palindrome?"
inputs = tokenizer(prompt, return_tensors="pt")
inputs=inputs.to(model.device)
outputs = model.generate(inputs["input_ids"], max_length=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


write a python program for checking palindrome?

palindrome = "abcd"
palindrome = "".join(reversed(palindrome))
print(palindrome)

# 2. Write a Python program to check whether a string is a palindrome or not.

def is_palindrome(s):
    return s == s[::-1]


def main():
    s = input("Enter a string: ")
    if is_palindrome(s):
        print(f"{s} is a palindrome")
    else:
        print(f"{s} is not a palindrome
